#Téléchargement des packages Python/spaCy

In [1]:
#!python -m spacy download fr_core_news_sm

Importation des librairies

In [2]:
import time
start = time.time()
import spacy
end = time.time()
print(end - start)
import re
import pandas as pd
import csv
from langdetect import detect

76.65170979499817


In [3]:
# Charger le fichier CSV dans un dataframe
df = pd.read_csv('GooglePlayReviews.csv')

# Fonction pour détecter la langue d'un commentaire
def detect_language(comment):
    try:
        lang = detect(comment)
        return lang
    except:
        return None

# Filtrer les commentaires en français
df_fr = df[df['commentaire'].apply(lambda x: detect_language(x)=='fr')]

df_fr.to_csv('Comments_Filtrés.csv', index=False, header = False)

Lecture du fichier csv

In [4]:
# open the CSV file using the csv module
with open('Comments_Filtrés.csv', newline='',  encoding='utf-8') as csvfile:
    # create a csv reader object
    reader = csv.reader(csvfile)
    # create an empty list to store the strings
    string_list = []
    # loop through each row in the CSV file
    for row in reader:
        # add the string to the list
        #if detect(row[0]) == 'fr':
        string_list.append(row[0])

Exemple de textes

In [5]:
#Exemple de texte
text = "Selon Wikipédia il l'appelle Laura 😊😊😊(hezignborzingiozrnoivznioznvionzo), La ponctuation est génial l’ensemble des signes qui, dans l’écrit, marquent les divisions et les liaisons des phrases et des membres 'de phrase'. Vous pouvez en apprendre plus sur \"la ponctuation\" en visitant ce site: https://www.larousse.fr/dictionnaires/francais/ponctuation/63717 parce que ce lien donne beaucoup d'informations intéressantes. 😊"
#text = "Fonctionnalités, salut car beau temps aujourd'hui"

Preprocessing

In [6]:
#Etapes du texte Preprocessing
nlp = spacy.load("fr_core_news_sm")
listStopWords = [str(x) for x in nlp.Defaults.stop_words]

Processing the text

In [7]:
def united(text):
    doc =  text.split(" ");
    # Itérer à travers chaque token dans le document
    for token in doc:

        #Remove explanations
        # Si le token est "parce que", "car" ou "puisque", supprimer tous les tokens suivants dans la phrase
        if token in ["parce", "car", "puisque"]:
            doc = doc[:token.index()]
            break
    
    # Supprimer tout ce qui est entre parenthèses
    sentence = ' '.join(doc)
    sentence = re.sub(r'\([^()]*\)', '', sentence)

    print(f"remove explanations : {sentence}")

    sentence = re.sub('http[s]?://\S+', '', sentence)

    print(f"remove links: {sentence}")


    # Regular expressions for phone numbers and email addresses
    quotes = r'\"[^\"]+\"'
    phone_regex = r"(?<!\d)(?:\d[ -/\\_\d]*){10}(?!\d)"
    email_regex = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"  # matches john.doe@example.com

    # Find phone numbers and email addresses in the sentence
    sentence = re.sub(quotes,"", sentence)

    print(f"remove links2: {sentence}")


    sentence = re.sub(phone_regex,"", sentence)
    sentence = re.sub(email_regex,"", sentence)
    sentence = re.sub(r'[^\w]', ' ', sentence)
    doc = sentence.split(" ")
    for word in doc:
        if word in listStopWords:
            doc.remove(word)
    
    sentence = ' '.join(doc)

    print(f"remove phone numbers and contacts: {sentence}")

    #Removal of ' and the char that precede it
    sentence = sentence.replace("'", " ")

    print(f"remove leopaul: {sentence}")

    words = sentence.split(" ")
    newSentence = ""
    for word in words:
        if len(word) > 1:
            newSentence += word + " "
    sentence = newSentence
    
    print(f"remove quotes : {sentence}")

    doc = nlp(sentence)
    #texte = doc.text
    text = ""
    for token in doc:
        if (token.pos_ == "PRON" or token.pos_ == "DET"):
            continue
        else:
            if not token.text.__contains__("'"):
                text += token.text
                text += " "
    sentence = text

    print(f"remove pronoum and determinant : {sentence}")


    doc = nlp(sentence)
    result = []
    for token in doc:
        if not (token.ent_type_ == "ORG" or token.ent_type_ == "PERSON" or token.text.istitle()):
            result.append(token.text)
    sentence =  " ".join(result)
    print(f"remove proper noun : {sentence}")

    return sentence

"""
for string in string_list:
    united_string = united(string)
    string_list_clean.append(united_string)
print(string_list_clean)
#run
#print(united(text))"""
united_string = united(string_list[0])
print(united_string)



remove explanations : Bonjour, Avis aux Amateurs et Professionnels, Depuis une dizaine de jours, et quelques sorties Course à pied, je n'ai plus accès aux données "Moyenne " dans la synthèse proposé par l'application : plus d'allure moyenne, plus de rythme moyen, et plus de vitesse moyenne, etc. Avez une explication et/ou solution. Sinon très bonne application, ou il manque un mode fractionné. Très ergonomique et très intuitive. Merci 😁
remove links: Bonjour, Avis aux Amateurs et Professionnels, Depuis une dizaine de jours, et quelques sorties Course à pied, je n'ai plus accès aux données "Moyenne " dans la synthèse proposé par l'application : plus d'allure moyenne, plus de rythme moyen, et plus de vitesse moyenne, etc. Avez une explication et/ou solution. Sinon très bonne application, ou il manque un mode fractionné. Très ergonomique et très intuitive. Merci 😁
remove links2: Bonjour, Avis aux Amateurs et Professionnels, Depuis une dizaine de jours, et quelques sorties Course à pied, j

#Part 2


In [8]:
united_string

'dizaine jours sorties pied accès données synthèse proposé application allure moyenne plus rythme moyen plus de vitesse moyenne explication solution bonne application ou manque mode fractionné ergonomique très intuitive'

In [9]:
# Define list of POS patterns to detect
patterns = [
    ("noun noun", ["NOUN", "NOUN"]),
    ("verb noun", ["VERB", "NOUN"]),
    ("adjective noun", ["ADJ", "NOUN"]),
    ("noun conjunction", ["NOUN", "CONJ"]),
    ("noun adjective", ["NOUN", "ADJ"]),
    ("noun noun noun", ["NOUN", "NOUN", "NOUN"]),
    ("verb pronoun noun", ["VERB", "PRON", "NOUN"]),
    ("verb noun noun", ["VERB", "NOUN", "NOUN"]),
    ("verb adjective noun", ["VERB", "ADJ", "NOUN"]),
    ("adjective adjective noun", ["ADJ", "ADJ", "NOUN"]),
    ("noun preposition noun", ["NOUN", "ADP", "NOUN"]),
    ("verb determiner noun", ["VERB", "DET", "NOUN"]),
    ("verb noun preposition noun", ["VERB", "NOUN", "ADP", "NOUN"]),
    ("adjective noun noun noun", ["ADJ", "NOUN", "NOUN", "NOUN"]),
    ("adjective conjunction adjective", ["ADJ", "CONJ", "ADJ"]),
    ("verb preposition adjective noun", ["VERB", "ADP", "ADJ", "NOUN"]),
    ("verb pronoun adjective noun", ["VERB", "PRON", "ADJ", "NOUN"]),
    ("noun conjunction noun noun", ["NOUN", "CONJ", "NOUN", "NOUN"]),
]

In [10]:
# Iterate through the reference patterns and check if there is a match in the sentence
for pattern, pos_tags_pattern in patterns:
    if len(pos_tags_pattern) != len(doc):
        continue  # Skip patterns with different number of words
    match = True
    for i in range(len(doc)):
        if pos_tags_pattern[i] != doc[i].pos_:
            match = False
            break
    if match:
        print(f"Detected pattern '{pattern}' in sentence '{united_string}'")
        break


NameError: name 'doc' is not defined

In [ ]:
# Define the input sentence
sentence = "The cat sat on the mat"

# Process the sentence with spaCy
doc = nlp(sentence)

# Iterate through the reference patterns and check if there is a match in the sentence
for pattern, pos_tags_pattern in patterns:
    if len(pos_tags_pattern) != len(doc):
        continue  # Skip patterns with different number of words
    match = True
    for i in range(len(doc)):
        if pos_tags_pattern[i] != doc[i].pos_:
            match = False
            break
    if match:
        print(f"Detected pattern '{pattern}' in sentence '{sentence}'")
        break


List of features

In [ ]:
sentenceByPattern = [[] for i in range((18))]

In [ ]:
sentence = "chat vert assoie sur canapé rouge"
doc = nlp(united_string)

Apply SAFE POS patterns

In [ ]:
for i in range(len(doc) - 3):
    if doc[i].pos_ == "NOUN":
        if doc[i + 1].pos_ == "CONJ" and doc[i + 2].pos_ == "NOUN":
            if doc[i + 3].pos_ == "NOUN":
                sentenceByPattern[17].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
                break
            else:
                sentenceByPattern[4].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                break
        elif doc[i + 1].pos_ == "PREP" and doc[i + 2].pos_ == "NOUN":
            sentenceByPattern[11].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
            break
        elif doc[i + 1].pos_ == "NOUN":
            if doc[i + 2].pos_ == "NOUN":
                sentenceByPattern[6].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                break
            else:
                sentenceByPattern[1].append(doc[i].text + " " + doc[i + 1].text)
                break
    elif doc[i].pos_ == "VERB":
        if doc[i + 1].pos_ == "PRON":
            if doc[i + 2].pos_ == "ADJ" and doc[i + 3].pos_ == "NOUN":
                sentenceByPattern[17].append(doc[i].text + " "+ doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
                break
            if doc[i + 2].pos_ == "NOUN":
                sentenceByPattern[7].append(doc[i].text +" " + doc[i + 1].text + " " + doc[i + 2].text)
                break
        elif doc[i + 1].pos_ == "PREP" and doc[i + 2].pos_ == "ADJ" and doc[i + 3].pos_ == "NOUN":
            sentenceByPattern[16].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
        elif doc[i + 1].pos_ == "NOUN" :
            if doc[i + 2].pos_ == "PREP" and doc[i + 3].pos_ == "NOUN":
                sentenceByPattern[13].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
                break
            elif doc[i + 2].pos_ == "NOUN":
                sentenceByPattern[8].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                break
            else:
                sentenceByPattern[2].append(doc[i].text + " " + doc[i + 1].text)
                break
        elif doc[i + 1].pos_ == "DET" and doc[i + 2].pos_ == "NOUN":
            sentenceByPattern[12].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
            break
        elif doc[i + 1].pos_ == "ADJ" and doc[i + 2].pos_ == "NOUN":
            sentenceByPattern[9].append(doc[i].text + " " + doc[i + 1].text + " " +doc[i + 2].text)
            break
    elif doc[i].pos_ == "ADJ" :
        if doc[i + 1].pos_ == "CONJ" and doc[i + 2].pos_ == "ADJ":
            sentenceByPattern[15].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
            break
        elif doc[i + 1].pos_ == "NOUN":
            if doc[i + 2].pos_ == "NOUN":
                if doc[i + 3].pos_ == "NOUN":
                    sentenceByPattern[14].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
                    break
                else:
                    sentenceByPattern[5].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                    break
            else:
                sentenceByPattern[3].append(doc[i].text + " " + doc[i + 1].text)
                break
        elif doc[i + 1].pos_ == "ADJ" and doc[i + 2].pos_ == "NOUN":
            sentenceByPattern[10].append(doc[i].text + " " + doc[i + 1].text +" " + doc[i + 2].text)
            break
    else:
        break

In [ ]:
sentenceByPattern

[[],
 ['dizaine jours', 'dizaine jours'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [ ]:
united_string

'dizaine jours sorties pied accès données synthèse proposé application allure moyenne plus rythme moyen plus de vitesse moyenne explication solution bonne application ou manque mode fractionné ergonomique très intuitive'

In [ ]:
def getPattern(doc):
    if(len(doc) < 4):
        return None
    for i in range(len(doc) - 3):
        if doc[i].pos_ == "NOUN":
            if doc[i + 1].pos_ == "CONJ" and doc[i + 2].pos_ == "NOUN":
                if doc[i + 3].pos_ == "NOUN":
                    sentenceByPattern[17].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
                    return getPattern(doc[i + 4:])
                else:
                    sentenceByPattern[4].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                    return getPattern(doc[i + 3:])
            elif doc[i + 1].pos_ == "PREP" and doc[i + 2].pos_ == "NOUN":
                sentenceByPattern[11].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                return getPattern(doc[i + 3:])
            elif doc[i + 1].pos_ == "NOUN":
                if doc[i + 2].pos_ == "NOUN":
                    sentenceByPattern[6].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                    return getPattern(doc[i + 3:])
                else:
                    sentenceByPattern[1].append(doc[i].text + " " + doc[i + 1].text)
                    return getPattern(doc[i + 2:])
        elif doc[i].pos_ == "VERB":
            if doc[i + 1].pos_ == "PRON":
                if doc[i + 2].pos_ == "ADJ" and doc[i + 3].pos_ == "NOUN":
                    sentenceByPattern[17].append(doc[i].text + " "+ doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
                    return getPattern(doc[i + 4:])
                if doc[i + 2].pos_ == "NOUN":
                    sentenceByPattern[7].append(doc[i].text +" " + doc[i + 1].text + " " + doc[i + 2].text)
                    return getPattern(doc[i + 3:])
            elif doc[i + 1].pos_ == "PREP" and doc[i + 2].pos_ == "ADJ" and doc[i + 3].pos_ == "NOUN":
                sentenceByPattern[16].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
                return getPattern(doc[i + 4:])
            elif doc[i + 1].pos_ == "NOUN" :
                if doc[i + 2].pos_ == "PREP" and doc[i + 3].pos_ == "NOUN":
                    sentenceByPattern[13].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
                    return getPattern(doc[i + 4:])
                elif doc[i + 2].pos_ == "NOUN":
                    sentenceByPattern[8].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                    return getPattern(doc[i + 3:])
                else:
                    sentenceByPattern[2].append(doc[i].text + " " + doc[i + 1].text)
                    return getPattern(doc[i + 2:])
            elif doc[i + 1].pos_ == "DET" and doc[i + 2].pos_ == "NOUN":
                sentenceByPattern[12].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                return getPattern(doc[i + 3:])
            elif doc[i + 1].pos_ == "ADJ" and doc[i + 2].pos_ == "NOUN":
                sentenceByPattern[9].append(doc[i].text + " " + doc[i + 1].text + " " +doc[i + 2].text)
                return getPattern(doc[i + 3:])
        elif doc[i].pos_ == "ADJ" :
            if doc[i + 1].pos_ == "CONJ" and doc[i + 2].pos_ == "ADJ":
                sentenceByPattern[15].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                return getPattern(doc[i + 3:])
            elif doc[i + 1].pos_ == "NOUN":
                if doc[i + 2].pos_ == "NOUN":
                    if doc[i + 3].pos_ == "NOUN":
                        sentenceByPattern[14].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
                        return getPattern(doc[i + 4:])
                    else:
                        sentenceByPattern[5].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                        return getPattern(doc[i + 3:])
                else:
                    sentenceByPattern[3].append(doc[i].text + " " + doc[i + 1].text)
                    return getPattern(doc[i + 2:])
            elif doc[i + 1].pos_ == "ADJ" and doc[i + 2].pos_ == "NOUN":
                sentenceByPattern[10].append(doc[i].text + " " + doc[i + 1].text +" " + doc[i + 2].text)
                return getPattern(doc[i + 3:])
        else:
            return getPattern(doc[i+1:])

In [ ]:
getPattern(nlp(united_string))

In [ ]:
for token in nlp(united_string):
    print(token.text, token.pos_, token.dep_)

dizaine NOUN ROOT
jours NOUN punct
sorties ADJ ROOT
pied ADJ ROOT
accès NOUN ROOT
données ADJ amod
synthèse NOUN amod
proposé VERB acl
application NOUN nmod
allure ADJ amod
moyenne NOUN advmod
plus ADV advmod
rythme NOUN obl:arg
moyen ADJ amod
plus ADV advmod
de ADP case
vitesse NOUN nmod
moyenne ADJ amod
explication NOUN amod
solution NOUN amod
bonne ADJ amod
application NOUN obj
ou CCONJ cc
manque ADJ conj
mode NOUN nsubj
fractionné VERB ROOT
ergonomique ADJ amod
très ADV advmod
intuitive ADJ advcl


In [ ]:
sentenceByPattern

[[],
 ['dizaine jours'],
 ['proposé application'],
 ['données synthèse', 'allure moyenne', 'bonne application', 'manque mode'],
 [],
 ['moyenne explication solution'],
 [],
 [],
 [],
 [],
 ['sorties pied accès'],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [ ]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
matched_sents = []  # Collect data of matched sentences to be visualized

def collect_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start:end]  # Matched span
    sent = span.sent  # Sentence containing matched span
    # Append mock entity for match in displaCy style to matched_sents
    # get the match span by ofsetting the start and end of the span with the
    # start and end of the sentence in the doc
    match_ents = [{
        "start": span.start_char - sent.start_char,
        "end": span.end_char - sent.start_char,
        "label": "MATCH",
    }]
    matched_sents.append({"text": sent.text, "ents": match_ents})

pattern = [{"LOWER": "facebook"}, {"LEMMA": "be"}, {"POS": "ADV", "OP": "*"},
           {"POS": "ADJ"}]
matcher.add("FacebookIs", [pattern], on_match=collect_sents)  # add pattern
doc = nlp("I'd say that Facebook is evil. – Facebook is pretty cool, right?")
matches = matcher(doc)

# Serve visualization of sentences containing match with displaCy
# set manual=True to make displaCy render straight from a dictionary
# (if you're not running the code within a Jupyer environment, you can
# use displacy.serve instead)
displacy.render(matched_sents, style="ent", manual=True)

In [ ]:
pattern = [{"POS": "ADJ"}, {"POS": "ADJ"} {"POS": "ADJ"}]

In [1]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
pattern1 = [{"POS": "NOUN"}, {"POS": "NOUN"}]
pattern2 = [{"POS": "VERB"}, {"POS": "NOUN"}]
pattern3 = [{"POS": "ADJ"}, {"POS": "NOUN"}]
pattern4 = [{"POS": "NOUN"}, {"POS": "CCONJ"}, {"POS": "NOUN"}]
pattern5 = [{"POS": "ADJ"}, {"POS": "NOUN"}, {"POS": "NOUN"}]
pattern6 = [{"POS": "NOUN"}, {"POS": "NOUN"}, {"POS": "NOUN"}]
pattern7 = [{"POS": "VERB"}, {"POS": "PRON"}, {"POS": "NOUN"}]
pattern8 = [{"POS": "VERB"}, {"POS": "NOUN"}, {"POS": "NOUN"}]
pattern9 = [{"POS": "VERB"}, {"POS": "ADJ"}, {"POS": "NOUN"}]
pattern10 = [{"POS": "ADJ"}, {"POS": "ADJ"}, {"POS": "NOUN"}]
pattern11 = [{"POS": "NOUN"}, {"POS": "ADP"}, {"POS": "NOUN"}]
pattern12 = [{"POS": "VERB"}, {"POS": "DET"}, {"POS": "NOUN"}]
pattern13 = [{"POS": "VERB"}, {"POS": "NOUN"}, {"POS": "ADP"}, {"POS": "NOUN"}]
pattern14 = [{"POS": "ADJ"}, {"POS": "NOUN"}, {"POS": "NOUN"}, {"POS": "NOUN"}]
pattern15 = [{"POS": "ADJ"}, {"POS": "CONJ"}, {"POS": "ADJ"}]
pattern16 = [{"POS": "VERB"}, {"POS": "ADP"}, {"POS": "ADJ"}, {"POS": "NOUN"}]
pattern17 = [{"POS": "VERB"}, {"POS": "PRON"}, {"POS": "ADJ"}, {"POS": "NOUN"}]
pattern18 = [{"POS": "NOUN"}, {"POS": "CCONJ"}, {"POS": "NOUN"}, {"POS": "NOUN"}]
matcher.add("18", [pattern18])
matcher.add("17", [pattern17])
matcher.add("16", [pattern16])
matcher.add("15", [pattern15])
matcher.add("14", [pattern14])
matcher.add("13", [pattern13])
matcher.add("12", [pattern12])
matcher.add("11", [pattern11])
matcher.add("10", [pattern10])
matcher.add("9", [pattern9])
matcher.add("8", [pattern8])
matcher.add("7", [pattern7])
matcher.add("6", [pattern6])
matcher.add("5", [pattern5])
matcher.add("4", [pattern4])
matcher.add("3", [pattern3])
matcher.add("2", [pattern2])
matcher.add("1", [pattern1])


doc = nlp("Song and artist album,  Create your greatest album, Choose from popular versions Choose from popular versions Fast system virus scanner Use depth of field Share an image Highlight with colors Super bright flashlight Perform intuitive gestures Enjoy group conversations Share your thoughts email chat history Live traffic conditions Phone or tablet Precise location Send message Group conversation, nice and warm, hot with orange")
matches = matcher(doc)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

#Remove duplicate and synonyms, noise

16743743820210141046 4 0 3 Song and artist
3049302120552557844 18 0 4 Song and artist album
5533571732986600803 1 2 4 artist album
6675425533856447351 17 6 10 Create your greatest album
602994839685422785 3 8 10 greatest album
4206877449096176963 16 11 15 Choose from popular versions
602994839685422785 3 13 15 popular versions
4206877449096176963 16 15 19 Choose from popular versions
602994839685422785 3 17 19 popular versions
602994839685422785 3 19 21 Fast system
2090661578966068036 5 19 22 Fast system virus
5533571732986600803 1 20 22 system virus
9798277639574861054 14 19 23 Fast system virus scanner
10999827425508017904 6 20 23 system virus scanner
5533571732986600803 1 21 23 virus scanner
15180167692696242062 2 23 25 Use depth
6349566914108460152 13 23 27 Use depth of field
2988402087957123519 11 24 27 depth of field
1124146173557384544 12 27 30 Share an image
6572986864102252890 10 33 36 Super bright flashlight
602994839685422785 3 34 36 bright flashlight
1819085394523955522 9 3